In [1]:
import pandas as pd

true_news = pd.read_csv('True.csv')
fake_news = pd.read_csv('Fake.csv')

In [2]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
true_news['valid'] = 1
fake_news['valid'] = 0

In [4]:
true_news = true_news.drop(['subject', 'date'], axis=1)
fake_news = fake_news.drop(['subject', 'date'], axis=1)

In [5]:
news_data = pd.concat([true_news, fake_news], ignore_index=True)
news_data = news_data.sample(frac=1)

In [6]:
import re
def remove_punctuations(text):
    return re.sub(r"[^\w\s]", "", text)
news_data[['title', 'text']] = news_data[['title', 'text']].apply(lambda x: x.str.lower())
news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_punctuations)
news_data.head()

<ipython-input-6-511c0eb6c4e8>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_punctuations)


,title,text,valid
34101,wow loose cannon hillary leaked major nuclear ...,is it not enough that hillary has compromised ...,0
36739,video trumps girls hammer msnbc after being un...,there s something about these sassy ladies tha...,0
18637,turkey opens military base in mogadishu to tra...,mogadishuankara reuters turkey opened its big...,1
8861,obama in shift says he will keep 8400 us troop...,washington reuters president barack obama cal...,1
3751,trump asked comey to end investigation of mich...,washington reuters us president donald trump ...,1


In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens_without_sw = [token for token in tokens if not token.lower() in stopwords.words('english')]
    return " ".join(tokens_without_sw)

news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_stopwords)
news_data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
<ipython-input-7-a733bd0ee2f5>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  news_data[['title', 'text']] = news_data[['title', 'text']].applymap(remove_stopwords)


,title,text,valid
34101,wow loose cannon hillary leaked major nuclear ...,enough hillary compromised national security u...,0
36739,video trumps girls hammer msnbc uninvited appe...,something sassy ladies makes makes want cheer ...,0
18637,turkey opens military base mogadishu train som...,mogadishuankara reuters turkey opened biggest ...,1
8861,obama shift says keep 8400 us troops afghanist...,washington reuters president barack obama call...,1
3751,trump asked comey end investigation michael fl...,washington reuters us president donald trump a...,1


In [8]:
from sklearn.model_selection import train_test_split
news_data['text'] = news_data['title'] + ' ' + news_data['text']
news_data = news_data.drop(['title'], axis=1)
X = news_data['text']
Y = news_data['valid']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [10]:
import joblib
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)

In [12]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print(f"accuracy score: {accuracy_score(y_test, y_pred)}")
print(f"precision score: {precision_score(y_test, y_pred)}")
print(f"recall score: {recall_score(y_test, y_pred)}")
print(f"f1 score: {f1_score(y_test, y_pred)}")
print(f"confusion matrix: {confusion_matrix(y_test, y_pred)}")

accuracy score: 0.9896436525612472
precision score: 0.9876288659793815
recall score: 0.9910344827586207
f1 score: 0.9893287435456111
confusion matrix: [[4576   54]
 [  39 4311]]
